## Packages, Constants and File paths

The dataset's audio clips are of 6 classes and stored in 6 folders corresponding to each speech command: 
- `ddyo`- `kkono` - `mu maaso` - `emabega` - `yimirira` - `unknown`

In [ ]:
from packages.utils import *

In [ ]:
from packages.utils import list_directory_contents

train_commands = list_directory_contents(train_data_dir, 'Train')
test_commands = list_directory_contents(test_data_dir, 'Test')

## Train and validation Datasets

In [ ]:
from packages.mfcc import load_train_dataset

train_mfcc_ds, val_mfcc_ds, mapping = load_train_dataset(train_json, BATCH_SIZE, VALIDATION_SPLIT)

## Test dataset

In [ ]:
# Function to load test dataset
from packages.mfcc import load_test_dataset

test_mfcc_ds, mapping = load_test_dataset(train_json, BATCH_SIZE)

# Model 1

### Input shape 

In [ ]:
example_spectrograms = next(iter(train_mfcc_ds))[0]
input_shape = example_spectrograms.shape[1:]

print('Input shape:', input_shape)
num_labels = len(mapping)

In [ ]:
print(f'Labels {mapping}')
print(f'Number of labels: {num_labels}')

In [ ]:
# Model 
from packages.model import model

model = model(input_shape, num_labels)

### Model Architecture

In [ ]:
model.summary()

### Compile and Train the model

In [ ]:
from packages.model import compile_and_train_model


history = compile_and_train_model(model, train_mfcc_ds, val_mfcc_ds)

### Plot Accuracy and Loss

In [ ]:
# Function to plot the training history
from packages.model import plot_training_history

plot_training_history(history)

### Evaluate the model performance

Run the model on the test set and check the model's performance:

In [ ]:
# Function to evaluate the model on the test dataset
from packages.model import evaluate_model

evaluate_model(model, test_mfcc_ds)

## Confusion matrix

In [ ]:
y_pred = model.predict(test_mfcc_ds)
y_pred = tf.argmax(y_pred, axis=1)
y_true = tf.concat(list(test_mfcc_ds.map(lambda s,lab: lab)), axis=0)
label_names_slice = ['ddyo', 'emabega', 'gaali', 'kkono', 'mumaaso', 'unknown', 'yimirira']

In [ ]:
# Function to plot the confusion matrix
from packages.model import plot_confusion_matrix

plot_confusion_matrix(y_true, y_pred, label_names_slice)

## save the Keras model

In [ ]:
KERAS_MODEL_PATH = "model/mfcc_model_1.keras"
model.save(KERAS_MODEL_PATH)

## Size of the keras model

In [ ]:
# Function to get the file size
from packages.model import get_model_size

keras_model_size = get_model_size(KERAS_MODEL_PATH, 'MB')